# Nettoyez et analysez votre jeu de données Nettoyez et analysez votre jeu de données 

## Nettoyer vos données avec python

### Detecter les erreurs

In [2]:
# import des librairies dont nous aurons besoin
import pandas as pd
import numpy as np
import re

In [3]:
# chargement et affichage des données
data = pd.read_csv('personnes.csv')
display(data)

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.49m
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
3,Marc,"marco23@example.com, mc23@supermail.eu",10/02/1978,France,1.65m
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m
5,Hanna,hanna2019@supermail.eu,01/01/1970,24,3.45m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


#### Valeurs manquantes

In [4]:
#Detecter les valeurs manquante (les NULL)
print(data.isnull().sum())

prenom            0
email             0
date_naissance    1
pays              1
taille            0
dtype: int64


#### Doublons

In [5]:
#Detecter les doublons
data.loc[data['email'].duplicated(keep=False),:]

,prenom,email,date_naissance,pays,taille
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m


### Traitez les erreurs : 

#### Theorie

`# Nouvelle dataframe :`

`data['nom_colonne'] = nouvelle_colonne`

`mask = # condition à vérifier pour cibler spécifiquement certaines lignes`

`data.loc[mask, 'ma_colonne'] = nouvelles_valeurs`

#### Traitez les pays :

In [6]:
display(data['pays'])

0           France
1              NaN
2    Côte d'ivoire
3           France
4       Madagascar
5               24
6            Bénin
Name: pays, dtype: object

- Le probleme etait que il y a la valeur 24 qui ne correspond pas a un pays
- Pour regler cette erreurs nous allons creer un dictionnaire ou nous allons mettre nos pays
- Nous alons donc verifier si le pays correspond ou non a un pays du dictionnaire
- Si il correspond alors tout est bon
- Si ça correspond pas alors on met un ***NaN***

In [7]:
VALID_COUNTRIES = ['France', 'Côte d\'ivoire', 'Madagascar', 'Bénin', 'Allemagne'
                  , 'USA']
mask = ~data['pays'].isin(VALID_COUNTRIES)
data.loc[mask, 'pays'] = np.NaN
display(data['pays'])

0           France
1              NaN
2    Côte d'ivoire
3           France
4       Madagascar
5              NaN
6            Bénin
Name: pays, dtype: object

On remarque que la valeur ***24*** qui n'etait pas dans le dictionnaire er devnu la valeur ***Nan***

#### Traitez les e-mails

In [8]:
data['email'] 

0                         leila@example.com
1                    samuel_329@example.com
2                  choupipoune@supermail.eu
3    marco23@example.com, mc23@supermail.eu
4                   helloworld@supermail.eu
5                    hanna2019@supermail.eu
6                    samuel_329@example.com
Name: email, dtype: object

Au tour des e-mails, maintenant ! Le problème avec cette colonne, c'est qu'il y a parfois 2 adresses e-mail par ligne. Nous ne souhaitons prendre que la première

In [9]:
data['email'] = data['email'].str.split(',', n=1, expand=True)[0]
display(data['email'])

0           leila@example.com
1      samuel_329@example.com
2    choupipoune@supermail.eu
3         marco23@example.com
4     helloworld@supermail.eu
5      hanna2019@supermail.eu
6      samuel_329@example.com
Name: email, dtype: object

Nous avons plus que un email par ligne

#### Traitez les tailles

In [10]:
display(data['taille'])

0    1.49m
1    1.67m
2    153cm
3    1.65m
4    1.34m
5    3.45m
6    1.45m
Name: taille, dtype: object

On remarque une taille impossible et un taille en cm

Ici nous decidons de garder seulement les taille en m. Et puis de donner les taille manquantes grace a la moyenne des autre tailles. Nous ne faisons pax la conversion cm -> m

In [11]:
data['taille'] = data['taille'].str[:-1]
data['taille'] = pd.to_numeric(data['taille'], errors='coerce')
display(data['taille'])

0    1.49
1    1.67
2     NaN
3    1.65
4    1.34
5    3.45
6    1.45
Name: taille, dtype: float64

In [12]:
data.loc[data['taille'].isnull(), 'taille'] = data['taille'].mean()
data['taille']=data['taille'].round(2)
display(data['taille'])

0    1.49
1    1.67
2    1.84
3    1.65
4    1.34
5    3.45
6    1.45
Name: taille, dtype: float64

On remarque que il rest la taille de 3.45 m 
On va essayrer de la traiter

In [24]:
data.loc[(data['taille']<=1.0) | (data['taille']>= 2.5),'taille'] = data['taille'].mean()
data['taille']=data['taille'].round(2)
display(data['taille'])

0    1.49
1    1.67
2    1.84
3    1.65
4    1.34
5    1.84
6    1.45
Name: taille, dtype: float64

On remarque que nous avons remplacer la valeur trop haute

#### Triez les dates

Nous allons spécifier un certain format de date, et transformer notre variable dans un type adéquat. Les formats non adequat vont etre transformer en NaN grace à ***coerce*** comme pour les tailles

In [26]:
data['date_naissance'] = pd.to_datetime(data['date_naissance'], format='%d/%m/%Y', errors='coerce')
display(data['date_naissance'])

0   1990-01-23
1   2001-09-20
2          NaT
3   1978-02-10
4   2008-03-05
5   1970-01-01
6          NaT
Name: date_naissance, dtype: datetime64[ns]

### Exercice chapitre 2

In [27]:
#importation des données
data = pd.read_csv('operations.csv')
data.head()

,date_operation,libelle,montant,solde_avt_ope,categ
0,2023-03-31,DON XX XX XX XX XX XX XX,-1.44,1515.25,AUTRE
1,2023-04-03,CARTE XX XX RAPT XX,-24.00,1513.81,TRANSPORT
2,2023-04-03,CARTE XX XX RAPT XX,-73.00,1489.81,TRANSPORT
3,2023-04-03,VIREMENT XX XX XX XX XX XX XX XX XX XX XX XX,676.00,1416.81,AUTRE
4,2023-04-03,VIREMENT XX XX XX XX XX XX,4.80,2092.81,AUTRE


<p>Instruction</p>
<blockquote> Plusieurs erreurs se sont glissées dans ce jeu de données. Votre mission, si toutefois vous l'acceptez, va être de les trouver et de proposer des solutions adéquates pour les gérer. Vous aurez besoin dans tous les cas de nettoyer votre fichier avant de passer à la suite !</blockquote>